In [1]:
import pandas as pd

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (HoverTool, ColumnDataSource, NumeralTickFormatter, DatetimeTickFormatter, RangeTool)
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.transform import jitter

In [5]:
LINK = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

In [6]:
quakes = pd.read_csv(LINK, parse_dates=['time', 'updated'])
quakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9132 entries, 0 to 9131
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   time             9132 non-null   datetime64[ns, UTC]
 1   latitude         9132 non-null   float64            
 2   longitude        9132 non-null   float64            
 3   depth            9132 non-null   float64            
 4   mag              9127 non-null   float64            
 5   magType          9127 non-null   object             
 6   nst              6427 non-null   float64            
 7   gap              7693 non-null   float64            
 8   dmin             6507 non-null   float64            
 9   rms              9131 non-null   float64            
 10  net              9132 non-null   object             
 11  id               9132 non-null   object             
 12  updated          9132 non-null   datetime64[ns, UTC]
 13  place            9

In [7]:
quakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-12-21 23:32:22.969000+00:00,36.226700,-115.312800,5.10,1.70,ml,12.0,187.95,0.30500,0.2654,...,2021-12-21 23:43:15.954000+00:00,"12 km N of Summerlin South, Nevada",earthquake,NaN,3.40,0.910,8.0,automatic,nn,nn
1,2021-12-21 23:08:54.500000+00:00,33.378667,-116.874333,6.28,0.08,ml,12.0,71.00,0.02683,0.1300,...,2021-12-21 23:43:28.964000+00:00,"3km NNW of Palomar Observatory, CA",earthquake,0.35,0.51,0.075,4.0,reviewed,ci,ci
2,2021-12-21 23:08:19.070000+00:00,40.299999,-124.544670,18.86,2.94,md,18.0,255.00,0.20030,0.1200,...,2021-12-21 23:36:54.875000+00:00,"22km W of Petrolia, CA",earthquake,2.01,0.57,0.050,9.0,automatic,nc,nc
3,2021-12-21 22:59:27.070000+00:00,33.378667,-116.873500,6.51,0.76,ml,31.0,42.00,0.02659,0.1800,...,2021-12-21 23:37:50.345000+00:00,"3km NNW of Palomar Observatory, CA",earthquake,0.23,0.41,0.094,13.0,reviewed,ci,ci
4,2021-12-21 22:58:48.020000+00:00,19.484300,-67.751000,29.00,3.45,md,20.0,229.00,1.13290,0.4500,...,2021-12-21 23:39:01.109000+00:00,"121 km NE of Punta Cana, Dominican Republic",earthquake,1.96,27.44,0.140,17.0,reviewed,pr,pr


In [8]:
quakes.shape

(9132, 22)

In [9]:
quakes['timeframe'] = quakes['time'].apply(lambda x: str(x)[0:10])
quakes['starttime'] = quakes['time'].apply(lambda x: str(x)[0:19])
quakes['updatetime'] = quakes['updated'].apply(lambda x: str(x)[0:19])

quakes.filter(regex="time")

,time,timeframe,starttime,updatetime
0,2021-12-21 23:32:22.969000+00:00,2021-12-21,2021-12-21 23:32:22,2021-12-21 23:43:15
1,2021-12-21 23:08:54.500000+00:00,2021-12-21,2021-12-21 23:08:54,2021-12-21 23:43:28
2,2021-12-21 23:08:19.070000+00:00,2021-12-21,2021-12-21 23:08:19,2021-12-21 23:36:54
3,2021-12-21 22:59:27.070000+00:00,2021-12-21,2021-12-21 22:59:27,2021-12-21 23:37:50
4,2021-12-21 22:58:48.020000+00:00,2021-12-21,2021-12-21 22:58:48,2021-12-21 23:39:01
...,...,...,...,...
9127,2021-11-22 00:23:41.561000+00:00,2021-11-22,2021-11-22 00:23:41,2021-11-27 22:41:43
9128,2021-11-22 00:15:23.260000+00:00,2021-11-22,2021-11-22 00:15:23,2021-11-22 18:46:27
9129,2021-11-22 00:12:06.350000+00:00,2021-11-22,2021-11-22 00:12:06,2021-11-22 01:28:11
9130,2021-11-22 00:02:17.320000+00:00,2021-11-22,2021-11-22 00:02:17,2021-11-22 16:46:16


In [10]:
quakes = quakes.loc[quakes['mag']>0]

In [11]:
quakes.isna().sum()

time                  0
latitude              0
longitude             0
depth                 0
mag                   0
magType               0
nst                2700
gap                1434
dmin               2471
rms                   1
net                   0
id                    0
updated               0
place                 0
type                  0
horizontalError    2174
depthError            0
magError           1913
magNst             1438
status                0
locationSource        0
magSource             0
timeframe             0
starttime             0
updatetime            0
dtype: int64

In [12]:
# completed cases only
quakes = quakes.dropna(how="any")
quakes.isna().sum()

time               0
latitude           0
longitude          0
depth              0
mag                0
magType            0
nst                0
gap                0
dmin               0
rms                0
net                0
id                 0
updated            0
place              0
type               0
horizontalError    0
depthError         0
magError           0
magNst             0
status             0
locationSource     0
magSource          0
timeframe          0
starttime          0
updatetime         0
dtype: int64

In [13]:
quakes.shape

(3914, 25)

In [14]:
quakes = quakes.sort_values("time").reset_index(drop=True)
quakes.nunique()

time               3914
latitude           3468
longitude          3556
depth              1698
mag                 367
magType               3
nst                  97
gap                 304
dmin               3303
rms                  57
net                   9
id                 3914
updated            3914
place              1990
type                  4
horizontalError     456
depthError          627
magError            612
magNst               79
status                2
locationSource        9
magSource             9
timeframe            30
starttime          3912
updatetime         3900
dtype: int64

In [15]:
quakes.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms,horizontalError,depthError,magError,magNst
count,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000,3914.000000
mean,36.603458,-114.381485,7.448839,1.317560,20.313746,108.183444,0.097471,0.122905,0.568720,2.656751,0.172706,11.291262
std,6.133049,13.261304,9.281991,0.775514,13.884363,61.041657,0.159699,0.085130,1.247263,6.937468,0.091389,12.167961
min,17.726000,-125.918833,-3.460000,0.010000,2.000000,13.000000,0.000550,0.000000,0.090000,0.100000,0.000000,1.000000
25%,33.821667,-122.123667,2.430000,0.750000,11.000000,63.000000,0.020855,0.060000,0.220000,0.410000,0.110000,4.000000
50%,36.971417,-117.603333,5.980000,1.160000,17.000000,89.000000,0.054130,0.110000,0.340000,0.610000,0.160000,8.000000
75%,38.831165,-115.094917,10.000000,1.800000,25.000000,140.000000,0.099675,0.170000,0.590000,1.236291,0.213000,14.000000
max,49.432333,-63.662300,184.000000,4.440000,174.000000,359.000000,2.190100,0.790000,54.500000,31.610000,0.774000,269.000000


In [16]:
quakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,horizontalError,depthError,magError,magNst,status,locationSource,magSource,timeframe,starttime,updatetime
0,2021-11-22 00:02:17.320000+00:00,41.792333,-112.377500,5.870000,0.74,md,9.0,87.0,0.134500,0.17,...,0.940000,3.060000,0.142000,8.0,reviewed,uu,uu,2021-11-22,2021-11-22 00:02:17,2021-11-22 16:46:16
1,2021-11-22 00:15:23.260000+00:00,49.389167,-120.509500,-0.370000,1.77,ml,12.0,185.0,0.581900,0.32,...,1.690000,31.610000,0.351628,5.0,reviewed,uw,uw,2021-11-22,2021-11-22 00:15:23,2021-11-22 18:46:27
2,2021-11-22 00:49:28.090000+00:00,48.617000,-122.399500,18.940000,1.40,ml,22.0,65.0,0.060150,0.35,...,0.550000,1.080000,0.109242,6.0,reviewed,uw,uw,2021-11-22,2021-11-22 00:49:28,2021-11-22 18:54:00
3,2021-11-22 01:06:34.816000+00:00,31.659525,-104.334754,7.571216,2.80,ml,31.0,60.0,0.118229,0.20,...,0.856934,1.273561,0.100000,16.0,reviewed,tx,tx,2021-11-22,2021-11-22 01:06:34,2021-11-29 03:29:58
4,2021-11-22 01:26:19.960000+00:00,34.328500,-118.502333,3.720000,1.28,ml,17.0,148.0,0.026000,0.16,...,0.460000,0.560000,0.110000,23.0,reviewed,ci,ci,2021-11-22,2021-11-22 01:26:19,2021-11-22 17:56:56
